In [1]:
import scipy.sparse as spr
import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np

import pyvista as pv
p = pv.BackgroundPlotter()

In [2]:
if coti.is_integer(type('3')):
    print(True)
else:
    print(False)

AttributeError: module 'pyoti.core' has no attribute 'is_integer'

In [4]:
import pyoti.core   as coti

In [4]:
tval = type(3)
%timeit coti.is_integer(tval)

103 ns ± 4.03 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [3]:
Th = fem.square( 2, 1, he = 1/100, element_order=2, structured = True, quads=False, save=False)

/Users/maristi7/opt/anaconda3/envs/pyoti/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


In [49]:
Th # -< Order 2

< mesh object with 861 nodes, 464 elements of types ( point1, line3, tri6 ) >

In [12]:
Th #-< Order 1

< mesh object with 20301 nodes, 40604 elements of types ( point1, line2, tri3 ) >

In [12]:
Th.group_names.keys()

dict_keys(['topleft', 'bottomleft', 'topright', 'bottomright', 'left', 'bottom', 'right', 'top', 'domain'])

In [13]:
Th.group_names['topleft']['members']

[{'types': array([15], dtype=int32),
  'tags': [array([1], dtype=uint64)],
  'nodes': array([1], dtype=uint64),
  'indices': [array([[0]], dtype=uint64)]}]

In [14]:
max(Th.y.real)

array([0.5])

In [15]:
fem.element[9]

< elbaseso object: 
 - Object status: ---------------- Started
 - Geometric type: --------------- elTriangle   
 - Kind of evaluation: ----------- elkindIso    
 - Number of derivatives: -------- 10
 - Number of dimensions: --------- 2
 - Number of basis: -------------- 6
 - Integration Order: ------------ 1
 - Array allocation: -------------- Not allocated
end elbaseso object >

In [16]:
Th.elements[2]['types']

array([2], dtype=int32)

In [17]:
Th.elements[2]['indices']

[array([[  0,   4,  59],
        [ 59,   4,  60],
        [ 59,  60,  58],
        ...,
        [230,  30,  31],
        [230,  31,  32],
        [ 32,  31,   3]], dtype=uint64)]

In [4]:
f = oti.sin(Th.x*Th.y)**2

In [6]:
f[np.int64(0),0]

0.2298

In [5]:
def test_funct(Thl,order=1,quads = False, nels = 10):
#     Thl = fem.square( 2, 1, he = 1/nels, element_order=order, structured = True, quads=quads, save=False)
    fl = oti.sin(Thl.x*Thl.y)**2
    return int2d(Thl, fl, intorder = order)
# f^ = sum f[j]* N[j]

In [7]:
def int2d(Th, f, intorder = 1, ndim_an  = 2):
    
    els = Th.elements[2]

    integral = oti.zero()
    
    fem.end_elements()
    
    #integrate 2D a fem array.
    for j in range(els['types'].size):

        element = fem.element[ els['types'][j] ]

        if not element.is_allocated():
            element.end()
            element.allocate(intorder)
            element.allocate_spatial(ndim_an,compute_Jinv = False)

            # Here the temporals shhould be created.
            fh = oti.zeros( ( element.nbasis, 1 ) )
            tmp1 = oti.zero(nip=element.nip)
            tmp2 = oti.zero()
        # end if

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):

            #  print('Element indices:',type(els['nodes'][j][i][0]))
            element.set_coordinates(Th.x,Th.y,Th.z,elm_nodes[i,:])
            element.compute_jacobian()

            for ii in range(element.nbasis):
                fh[ii,0] = f[ elm_nodes[i,ii], 0]
            #end for 
            
            oti.dot_product(fh,element.N,out=tmp1)
            oti.gauss_integrate(tmp1,element.w_dJ, out=tmp2)
            oti.sum(tmp2,integral, out=integral)
            
#             integral = integral + oti.gauss_integrate(oti.dot_product(fh,element.N),element.w_dJ)
            
            
#             integral = integral + oti.dot_product( fh, oti.gauss_integrate( element.N, element.w_dJ) )

        # end for 
    # end for.
    
    fem.end_elements()
    
    return integral


In [8]:
# TRI3-> 500 ms ± 7.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# %timeit int2d(Th, f, intorder=2)
%timeit int2d(Th, f, intorder=2)

2.71 s ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
fem.op_int2d(Th, f,intorder=2).real

0.053916929526403916

In [6]:
%timeit fem.op_int2d(Th, f, intorder = 2 )

782 ms ± 8.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
num = np.int64(3)
isinstance(num,np.int)
np.isin(3)

TypeError: _isin_dispatcher() missing 1 required positional argument: 'test_elements'

# 

In [22]:
num = np.uint64(3)

%timeit type_val = type(num)#;(type_val==int or type_val==np.uint64 or type_val==np.int64)
%timeit isinstance(num,(np.uint64))
isinstance(num,(int,np.uint64))

type(type(int))

104 ns ± 0.354 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
130 ns ± 0.824 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


True

In [44]:
order=2
quads = False
nels = 100
Thl = fem.square( 2, 1, he = 1/nels, element_order=order, structured = True, quads=quads, save=False)
%timeit test_funct(Thl,order=order, quads = quads, nels = 300)

1.06 s ± 35.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
els = Th.elements[2]

intorder = 1
ndim_an  = 2

# element = fem.tri3
# element.end()
# element.allocate(intorder)
# element.allocate_spatial(ndim_an,compute_Jinv = True)

integral = 0


#integrate 2D a fem array.
for j in range(els['types'].size):

    element = fem.element[ els['types'][j] ]
    
    if not element.is_allocated():
        element.end()
        element.allocate(intorder)
        element.allocate_spatial(ndim_an,compute_Jinv = True)

        # Here the temporals shhould be created.
        fh = oti.zeros( ( element.nbasis,1) )
    # end if 
    
    elm_nodes = els['indices'][j]
    
    for i in range(elm_nodes.shape[0]):
        
        #  print('Element indices:',type(els['nodes'][j][i][0]))
        element.set_coordinates(Th.x,Th.y,Th.z,elm_nodes[i,:])
        element.compute_jacobian()
        
        for ii in range(element.nbasis):
            fh[ii,0] = f[ int(elm_nodes[i,ii]),0]
        #end for 
        
        # integral = integral + oti.gauss_integrate(oti.dot_product(fh,element.N),element.w_dJ)
        integral = integral + oti.dot_product( fh, oti.gauss_integrate( element.N, element.w_dJ) )
        
    # end for 
    
# end for.
fem.end_elements()
integral.real

0.05349579666750341

In [20]:
els['types']

array([3], dtype=int32)

In [16]:
not element.is_initialized()

False

In [60]:
max(f.real)

array([0.22984885])

In [10]:
grid = Th.to_pv(dims=2,pd=[f.real],pd_names=["function"])

p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=16,cmap='jet')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7fe8c8ed82e8

In [51]:
Th.elements[2]

{'types': array([3], dtype=int32),
 'tags': [array([ 305,  306,  307, ..., 5302, 5303, 5304], dtype=uint64)],
 'nodes': [array([[   0,    4,  300,  299],
         [ 299,  300,  301,  298],
         [ 298,  301,  302,  297],
         ...,
         [5148,  149,  150, 5149],
         [5149,  150,  151, 5150],
         [5150,  151,    3,  152]], dtype=uint64)]}